# Validation Set 1.2: diffuPy + PathMe  

In [1]:
import os
import itertools

import time
import copy

dir_path = os.path.dirname(os.path.realpath('__file__'))

In [2]:
from openpyxl import load_workbook
from collections import defaultdict
import networkx as nx

In [3]:
import pybel
import pybel_tools as pbt

from pybel import BELGraph
from pybel.constants import RELATION, ANNOTATIONS
from pybel.dsl import Abundance, BiologicalProcess, CentralDogma, ListAbundance, Reaction
from pybel.struct import get_subgraph_by_annotation_value
from pybel.struct.summary import count_functions

pybel.get_version()

'0.13.2'

In [4]:
from pathme.constants import REACTOME_BEL, KEGG_BEL, WIKIPATHWAYS_BEL, PATHME_DIR
from pathme.export_utils import to_gml

In [5]:
from diffuPy.diffuse import diffuse
from diffuPy.kernels import regularised_laplacian_kernel
from diffuPy.matrix import Matrix, LaplacianMatrix
from diffuPy.input_mapping import generate_categoric_input_from_labels, get_mapping, get_mapping_subsets, get_mapping_two_dim_subsets
from diffuPy.validation_sets_utils import parse_set1
from diffuPy.utils import print_dict_dimentions, get_labels_set_from_dict, reduce_dict_dimention
from diffuPy.views import show_heatmap

from diffuPy.miscellaneous import get_label_list_graph


In [6]:
# TODO
import nbimporter
from global_database_comparison import get_set_database, calculate_database_sets_as_dict

Importing Jupyter notebook from global_database_comparison.ipynb


## 1. Load Data Set 1: Input Scores

In [7]:
dataset1_labels_by_omics = parse_set1(os.path.join(dir_path, 'validation', 'set1.xlsx'))

In [8]:
print_dict_dimentions(dataset1_labels_by_omics, 'Dataset1 imported labels:')

Dataset1 imported labels:
Total number of genes: 4941  
Total number of micrornas: 100  
Total number of metabolite: 21  
Total: 5062 


In [9]:
dataset1_all_labels = get_labels_set_from_dict(dataset1_labels_by_omics)

## 2. Load Backrgound Graph Universe

### 2.1. PathMeUniverse import

##### Database argument in node data

In [10]:
pathme_universe_graph_no_flatten = pybel.from_pickle(os.path.join(PATHME_DIR, 'universe', 'pathme_universe_bel_graph_no_flatten.bel.pickle'))


In [11]:
pathme_universe_graph_no_flatten.summarize()

PathMe Universe v1.0.0
Number of Nodes: 49697
Number of Edges: 104728
Number of Citations: 7706
Number of Authors: 0
Network Density: 4.24E-05
Number of Components: 7018
Number of Warnings: 0


##### Check isolates / Unconnected nodes

In [12]:
nx.number_of_isolates(pathme_universe_graph_no_flatten)

6534

In [13]:
pathme_universe_graph_no_flatten.remove_nodes_from({
    node
    for node in nx.isolates(pathme_universe_graph_no_flatten)
})

In [14]:
nx.number_of_isolates(pathme_universe_graph_no_flatten)

0

##### Export to gml

In [15]:
to_gml(pathme_universe_graph_no_flatten, os.path.join(PATHME_DIR, 'pathme_universe_graph_no_flatten.gml'))

In [16]:
nx_pathme_universe_graph_no_flatten = nx.read_gml(os.path.join(PATHME_DIR, 'pathme_universe_graph_no_flatten.gml'))

In [17]:
nx.info(nx_pathme_universe_graph_no_flatten)

'Name: \nType: MultiDiGraph\nNumber of nodes: 43105\nNumber of edges: 104728\nAverage in degree:   2.4296\nAverage out degree:   2.4296'

In [18]:
nx.number_of_isolates(nx_pathme_universe_graph_no_flatten)

0

### 2.2. Get labels subsets by entity type (omic) and database

##### PathMe package subsets

In [19]:
def get_labels_by_db_and_omic_from_pathme(databases):
    db_entites = defaultdict(dict)
    entites_db = defaultdict(lambda: defaultdict(set))
    
    for db in databases:
        genes, mirna, metabolites, bps = get_set_database(db)
        db_entites[db] = {'genes': genes, 'mirna': mirna, 'metabolites': metabolites, 'bps': bps}
        
        for entity_type, entities in db_entites[db].items():
            entites_db[entity_type][db] = entities
            
    return db_entites, entites_db

In [ ]:
bg_labels_from_pathme_by_db_and_omic, bg_labels_from_pathme_by_omic_and_db = get_labels_by_db_and_omic_from_pathme(['reactome', 'kegg', 'wikipathways'])

print_dict_dimentions(bg_labels_from_pathme_by_db_and_omic, 'Entities in PathMe package :')



Entities in PathMe package :
Total number of reactome: genes(8653), mirna(13), metabolites(2712), bps(2219),  
Total number of kegg: genes(7656), mirna(149), metabolites(4421), bps(418),  
Total number of wikipathways: genes(3361), mirna(91), metabolites(657), bps(135),  
Total: 30485 


In [ ]:
bg_labels_from_pathme_by_omic = reduce_dict_dimention(bg_labels_from_pathme_by_omic_and_db)
mirnas = bg_labels_from_pathme_by_omic['mirna']

#####  PathMeUniverse Graph Subsets

In [ ]:
def get_labels_by_db_and_omic_from_graph(graph):
    
    db_subsets_pmu = defaultdict(set)
    db_entites_pmu = defaultdict(dict)

    #entity_type_map = {'Gene':'genes', 'mirna_nodes':'mirna', 'Abundance':'metabolites', 'BiologicalProcess':'bps'}

    for u, v, k in graph.edges(keys=True):
        if ANNOTATIONS not in graph[u][v][k]:
            continue
            
        if 'database' not in graph[u][v][k][ANNOTATIONS]:
            continue
            
        for database in graph[u][v][k][ANNOTATIONS]['database']:
            db_subsets_pmu[database].add(u)
            db_subsets_pmu[database].add(v)
        
    for database, nodes in db_subsets_pmu.items():
        db_entites_pmu[database] = calculate_database_sets_as_dict(nodes, database)
        
    return db_entites_pmu, db_subsets_pmu

In [ ]:
bg_labels_from_pathmeuniverse_by_db_and_omic, bg_labels_from_pathmeuniverse_by_omic_and_db =  get_labels_by_db_and_omic_from_graph(pathme_universe_graph_no_flatten)

print_dict_dimentions(bg_labels_from_pathmeuniverse_by_db_and_omic, 'Entities in PathMeUniverse :')


Entities in PathMeUniverse :
Total number of kegg: gene_nodes(6037), mirna_nodes(122), metabolite_nodes(1753), bp_nodes(112),  
Total number of reactome: gene_nodes(8634), mirna_nodes(13), metabolite_nodes(2681), bp_nodes(0),  
Total number of wikipathways: gene_nodes(2555), mirna_nodes(91), metabolite_nodes(634), bp_nodes(135),  
Total: 22767 


### 2.3. Get subgraphs subsets by database

In [ ]:
kegg_pathme_graph_no_flatten = get_subgraph_by_annotation_value(pathme_universe_graph_no_flatten, 'database', 'kegg')
reactome_pathme_universe_graph_no_flatten = get_subgraph_by_annotation_value(pathme_universe_graph_no_flatten, 'database', 'reactome')
wikipathways_pathme_universe_graph_no_flatten = get_subgraph_by_annotation_value(pathme_universe_graph_no_flatten, 'database', 'wikipathways')


##### KEGG

In [ ]:
kegg_pathme_graph_no_flatten.summarize()
count_functions(kegg_pathme_graph_no_flatten)

None vNone
Number of Nodes: 13162
Number of Edges: 35122
Number of Citations: 2
Number of Authors: 0
Network Density: 2.03E-04
Number of Components: 65
Number of Warnings: 0


Counter({'Composite': 1596,
         'Complex': 405,
         'Gene': 6159,
         'Abundance': 1753,
         'Reaction': 3137,
         'BiologicalProcess': 112})

### Background  Matrix

In [ ]:
then = time.time()

In [ ]:
background_mat = regularised_laplacian_kernel(kegg_pathme_graph_no_flatten)

/Users/jmarinllao/Documents/CREB/diffuPy/src/diffuPy/miscellaneous.py:19: UserWarning: Graph must be undirected, so it is converted to undirected.
  warnings.warn('Graph must be undirected, so it is converted to undirected.')
Node name nor id not labeled: composite(p(HGNC:AKT1), p(HGNC:AKT2), p(HGNC:AKT3))
Node name nor id not labeled: composite(p(HGNC:TSC1), p(HGNC:TSC2))
Node name nor id not labeled: composite(p(HGNC:NFKB1), p(HGNC:RELA))
Node name nor id not labeled: complex(p(HGNC:CHUK), p(HGNC:IKBKB), p(HGNC:IKBKG))
Node name nor id not labeled: composite(p(HGNC:NOS1), p(HGNC:NOS2), p(HGNC:NOS3))
Node name nor id not labeled: composite(p(HGNC:MAPK1), p(HGNC:MAPK3))
Node name nor id not labeled: composite(p(HGNC:ADCY1), p(HGNC:ADCY2), p(HGNC:ADCY3), p(HGNC:ADCY4), p(HGNC:ADCY5), p(HGNC:ADCY6), p(HGNC:ADCY7), p(HGNC:ADCY8), p(HGNC:ADCY9))
Node name nor id not labeled: composite(p(HGNC:FOXO1), p(HGNC:FOXO3))
Node name nor id not labeled: composite(p(HGNC:AKT1S1), p(HGNC:MTOR), p(HGNC

Node name nor id not labeled: composite(p(HGNC:PCYT1A), p(HGNC:PCYT1B))
Node name nor id not labeled: composite(p(HGNC:IRS1), p(HGNC:IRS2), p(HGNC:IRS4))
Node name nor id not labeled: composite(p(HGNC:MAP2K1), p(HGNC:MAP2K2))
Node name nor id not labeled: composite(p(HGNC:ARAF), p(HGNC:BRAF), p(HGNC:RAF1))
Node name nor id not labeled: composite(p(HGNC:BRAF), p(HGNC:RAF1))
Node name nor id not labeled: composite(p(HGNC:SOS1), p(HGNC:SOS2))
Node name nor id not labeled: composite(p(HGNC:HRAS), p(HGNC:KRAS), p(HGNC:MRAS), p(HGNC:NRAS), p(HGNC:RRAS), p(HGNC:RRAS2))
Node name nor id not labeled: composite(p(HGNC:CRK), p(HGNC:CRKL))
Node name nor id not labeled: composite(p(HGNC:FTH1), p(HGNC:FTL))
Node name nor id not labeled: complex(p(HGNC:BCL2), p(HGNC:BCL2L1))
Node name nor id not labeled: composite(p(HGNC:"BUB1B-PAK6"), p(HGNC:PAK1), p(HGNC:PAK2), p(HGNC:PAK3), p(HGNC:PAK4), p(HGNC:PAK5), p(HGNC:PAK6))
Node name nor id not labeled: complex(composite(p(HGNC:WASF1), p(HGNC:WASF2), p(HGN

Node name nor id not labeled: complex(composite(p(HGNC:FZD1), p(HGNC:FZD10), p(HGNC:FZD2), p(HGNC:FZD3), p(HGNC:FZD4), p(HGNC:FZD5), p(HGNC:FZD6), p(HGNC:FZD7), p(HGNC:FZD8), p(HGNC:FZD9)), composite(p(HGNC:LRP5), p(HGNC:LRP6)))
Node name nor id not labeled: composite(p(HGNC:TGFB1), p(HGNC:TGFB2), p(HGNC:TGFB3))
Node name nor id not labeled: complex(p(HGNC:TGFBR1), p(HGNC:TGFBR2))
Node name nor id not labeled: composite(p(HGNC:TGFBR1), p(HGNC:TGFBR2))
Node name nor id not labeled: composite(p(HGNC:COL1A1), p(HGNC:COL1A2), p(HGNC:COL3A1), p(HGNC:COL4A1), p(HGNC:COL4A2), p(HGNC:COL4A3), p(HGNC:COL4A4), p(HGNC:COL4A5), p(HGNC:COL4A6))
Node name nor id not labeled: composite(p(HGNC:BMPR1A), p(HGNC:BMPR1B), p(HGNC:BMPR2))
Node name nor id not labeled: composite(p(HGNC:CTNNA1), p(HGNC:CTNNA2), p(HGNC:CTNNA3))
Node name nor id not labeled: composite(p(HGNC:ACTN1), p(HGNC:ACTN4))
Node name nor id not labeled: composite(p(HGNC:YWHAB), p(HGNC:YWHAE), p(HGNC:YWHAG), p(HGNC:YWHAH), p(HGNC:YWHAQ), 

Node name nor id not labeled: rxn(reactants(a(CHEBI:"acetyl-CoA")), products(a(CHEBI:"coenzyme A")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"coenzyme A")), products(a(CHEBI:"butyryl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"coenzyme A")), products(a(CHEBI:"acetyl-CoA")))
Node name nor id not labeled: composite(p(HGNC:HMGCS1), p(HGNC:HMGCS2))
Node name nor id not labeled: composite(p(HGNC:OXCT1), p(HGNC:OXCT2))
Node name nor id not labeled: composite(p(HGNC:HMGCL), p(HGNC:HMGCLL1))
Node name nor id not labeled: composite(p(HGNC:ACACA), p(HGNC:ACACB))
Node name nor id not labeled: composite(p(HGNC:PCCA), p(HGNC:PCCB))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"propionyl-CoA")), products(a(CHEBI:"(S)-methylmalonyl-CoA")))
Node name nor id not labeled: composite(p(HGNC:ACSS1), p(HGNC:ACSS2))
Node name nor id not labeled: composite(p(HGNC:SHMT1), p(HGNC:SHMT2))
Node name nor id not labeled: rxn(reactants(a(CHEBI:glycine)), products(a(CHEBI:"L-serine

Node name nor id not labeled: composite(p(HGNC:TARS), p(HGNC:TARS2), p(HGNC:TARSL2))
Node name nor id not labeled: composite(p(HGNC:SARS), p(HGNC:SARS2))
Node name nor id not labeled: composite(p(HGNC:HK1), p(HGNC:HK2), p(HGNC:HK3), p(HGNC:HKDC1))
Node name nor id not labeled: composite(p(HGNC:ALDOA), p(HGNC:ALDOB), p(HGNC:ALDOC))
Node name nor id not labeled: composite(p(HGNC:TKT), p(HGNC:TKTL1), p(HGNC:TKTL2))
Node name nor id not labeled: composite(p(HGNC:AGXT), p(HGNC:AGXT2))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"3-hydroxypyruvic acid")), products(a(CHEBI:"2-hydroxy-3-oxopropanoic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"4-hydroxy-2-oxoglutaric acid")), products(a(CHEBI:"glyoxylic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"4-hydroxy-2-oxoglutaric acid")), products(a(CHEBI:"pyruvic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"glyoxylic acid")), products(a(CHEBI:"4-hydroxy-2-oxoglutaric acid")))
Node name nor id 

...

Node name nor id not labeled: rxn(reactants(a(CHEBI:"2-Polyprenyl-3-methyl-5-hydroxy-6-methoxy-1,4-benzoquinone")), products(a(CHEBI:"2-Polyprenyl-3-methyl-6-methoxy-1,4-benzoquinone")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"2-Polyprenyl-3-methyl-5-hydroxy-6-methoxy-1,4-benzoquinone")), products(a(CHEBI:ubiquinones)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:ubiquinones)), products(a(CHEBI:"2-Polyprenyl-3-methyl-5-hydroxy-6-methoxy-1,4-benzoquinone")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"5beta-pregnane-3,20-dione")), products(a(CHEBI:progesterone)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"3alpha-hydroxy-5beta-pregnan-20-one")), products(a(CHEBI:"5beta-pregnane-3,20-dione")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:estriol)), products(a(CHEBI:"17beta-estradiol")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"dihydroxyacetone phosphate")), products(a(CHEBI:dihydroxyacetone)))
Node name nor id not labeled: rxn(rea

Node name nor id not labeled: rxn(reactants(a(CHEBI:"decanoic acid")), products(a(CHEBI:"decanoyl-[acyl-carrier protein]")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"dodecanoic acid")), products(a(PUBCHEM:7620)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:tetradecanoate)), products(a(CHEBI:"myristoyl-[acyl-carrier protein]")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"2-phospho-D-glyceric acid")), products(a(CHEBI:"2,3-bisphospho-D-glyceric acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"4-aminobutanal")), products(a(CHEBI:putrescine)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"N-acetyl-D-glucosamine 6-phosphate")), products(a(CHEBI:"N-acetyl-D-glucosamine")))
Node name nor id not labeled: rxn(reactants(a(PUBCHEM:254741470)), products(a(PUBCHEM:254741469)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"5-guanidino-2-oxopentanoic acid")), products(a(CHEBI:"D-arginine")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"

Node name nor id not labeled: rxn(reactants(a(CHEBI:"pyruvic acid")), products(a(CHEBI:"(R)-lactic acid")))
Node name nor id not labeled: rxn(reactants(a(PUBCHEM:163311981)), products(a(CHEBI:"Ethylmalonyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"propionyl-CoA")), products(a(PUBCHEM:254741472)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"but-2-enoyl-CoA")), products(a(CHEBI:"glutaryl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"2-hydroxyglutaric acid")), products(a(CHEBI:"2-oxoglutaric acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"cis-aconitic acid")), products(a(CHEBI:"itaconic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"itaconic acid")), products(a(CHEBI:"cis-aconitic acid")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"S-(hydroxymethyl)glutathione")), products(a(CHEBI:"S-formylglutathione")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"S-formylglutathione")), products(a(CHEBI:"S-(hydrox

Node name nor id not labeled: rxn(reactants(a(CHEBI:proton)), products(a(CHEBI:glycine)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"L-glutamic acid")), products(a(CHEBI:glycine)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"D-glyceraldehyde 3-phosphate")), products(a(CHEBI:"D-fructofuranose 1,6-bisphosphate")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"linoleoyl-CoA")), products(a(CHEBI:"gamma-linolenoyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"gamma-linolenoyl-CoA")), products(a(CHEBI:"linoleoyl-CoA")))
Node name nor id not labeled: rxn(reactants(a(PUBCHEM:254741325)), products(a(CHEBI:"1-alkyl-2-acyl-sn-glycerol")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"1-alkyl-2-acyl-3-(3-O-sulfo-beta-D-galactosyl)-sn-glycerol")), products(a(PUBCHEM:254741325)))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"5-phospho-beta-D-ribosylamine")), products(a(CHEBI:"L-glutamine")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"5

Node name nor id not labeled: rxn(reactants(a(CHEBI:"Citalopram aldehyde")), products(a(CHEBI:"Citalopram propionic acid")))
Node name nor id not labeled: rxn(reactants(composite(a(CHEBI:felbamate), a(CHEBI:felbamate))), products(a(CHEBI:"p-Hydroxyfelbamate")))
Node name nor id not labeled: rxn(reactants(composite(a(CHEBI:felbamate), a(CHEBI:felbamate))), products(a(CHEBI:"2-Hydroxyfelbamate")))
Node name nor id not labeled: rxn(reactants(composite(a(CHEBI:codeine), a(PUBCHEM:7847263))), products(a(CHEBI:norcodeine)))
Node name nor id not labeled: rxn(reactants(composite(a(CHEBI:carbamazepine), a(CHEBI:carbamazepine))), products(a(CHEBI:"3-Hydroxycarbamazepine")))
Node name nor id not labeled: rxn(reactants(a(CHEBI:"2-Hydroxycarbamazepine")), products(a(CHEBI:"5H-dibenzo[b,f]azepin-2-ol")))
Node name nor id not labeled: rxn(reactants(composite(a(CHEBI:"valproic acid"), a(CHEBI:"valproic acid"))), products(a(CHEBI:"3-Hydroxyvalproic acid")))
Node name nor id not labeled: rxn(reactants(a

In [ ]:
now = time.time()
print("It took: ", now-then, " seconds")

### Row labels

In [ ]:
background_labels = background_mat.rows_labels

In [ ]:
mirnas = dataset1_labels_by_omics['micrornas']

## 3. Dataset label mapping to PathMeEntities

### General mapping

In [ ]:
all_labels_mapping = get_mapping(dataset1_all_labels, background_labels, title = 'Global mapping: ', mirnas=mirnas, print_percentage = True)


### Mapping by subsets

#### Mapping by entity type/omic

In [ ]:
mapping_by_entity, _, _ = get_mapping_subsets(dataset1_labels_by_omics, background_labels, 'entity type/omic', mirnas=mirnas)


In [ ]:
micrornas_mapping = mapping_by_entity['micrornas'][0]
micrornas_mapping

#### Mapping by database

In [ ]:
background_entites_by_db = {db : set(itertools.chain.from_iterable(entities.values())) for db, entities in bg_labels_from_pathmeuniverse_by_db_and_omic.items()}
mapping_by_db, _, _ = get_mapping_subsets(background_entites_by_db, dataset1_all_labels, 'database', mirnas=mirnas, submapping = background_labels, mirnas_mapping = micrornas_mapping, percentage_reference_labels = True)


#### Mapping by entity type and database

In [ ]:
mapping_by_database_and_entity, total_percentage, total_dimention = get_mapping_two_dim_subsets(bg_labels_from_pathmeuniverse_by_db_and_omic, dataset1_all_labels, relative_statistics = dataset1_labels_by_omics, mirnas_mapping = micrornas_mapping)


### Mapping descriptive view

In [ ]:
# TODO

all_count = []
all_percentage = []

entity_type_map = {'metabolite_nodes': 'metabolite', 'mirna_nodes': 'micrornas', 'gene_nodes': 'genes',
                    'bp_nodes': 'bps'}

for db_name, entities_by_type in mapping_by_database_and_entity.items():
    db_count = []
    db_percentage = []

    for entity_type, entities_tupple in entities_by_type[0].items():
        db_count.append(len(entities_tupple[0]))
        db_percentage.append(entities_tupple[1])
        
        
    all_count.append(db_count)
    all_percentage.append(db_percentage)

db_count = []
db_percentage = []
for entity, entities_tupple in mapping_by_entity.items():
    db_count.append(len(entities_tupple[0]))
    db_percentage.append(entities_tupple[1])
    


In [ ]:
import numpy as np

databases = list(mapping_by_database_and_entity.keys())
databases.append('arreu')


entity_types = mapping_by_database_and_entity['kegg'][0].keys()

entity_number = np.array(all_percentage)
entity_count = np.array(all_count)

In [ ]:
databases = mapping_by_database_and_entity.keys()
entity_types = mapping_by_database_and_entity['kegg'][0].keys()

entity_number = np.array(all_percentage)
entity_count = np.array(all_count)

In [ ]:
show_heatmap(entity_number, entity_count, databases, entity_types)

## 4. Score Diffusion with diffuPy: Dataset as input + PathMe as background graph

### Input elements: Dataset input + Background matrix

#### Input vector/matrix construction

In [ ]:
input_mats_by_entity_type = {entity_type: generate_categoric_input_from_labels(all_labels_mapping, ['Dataset 1'], background_mat) for entity_type, map_value in mapping_by_entity.items()}


In [ ]:
input_mat = generate_categoric_input_from_labels(all_labels_mapping, ['Dataset 1'], background_mat)

In [ ]:
input_mat_raw = copy.copy(input_mat)
background_mat_raw = copy.copy(background_mat)

input_mat_z = copy.copy(input_mat)
background_mat_z = copy.copy(background_mat)

### Compute diffusion scores

#### Raw scores

In [ ]:
then = time.time()

In [ ]:
raw_scores = diffuse(input_mat_raw, 'ml', K = background_mat_raw)

In [ ]:
now = time.time()
print("It took: ", now-then, " seconds")

#### Normalized z-scores

In [ ]:
then = time.time()

In [ ]:
z_scores = diffuse(input_mat_z, 'ml', K = background_mat_z, z = True)

In [ ]:
now = time.time()
print("It took: ", now-then, " seconds")

## 5. Validation metrics

In [ ]:
len(input_mat.mat)

In [ ]:
import sklearn as skl

In [ ]:
skl.metrics.roc_auc_score(input_mat_raw, z_scores)